# Unsupervised Anomaly Detection Notebook
This notebook loads test.csv and trains Isolation Forest, One-Class SVM, and AutoEncoder models.

In [1]:
!pip install kaggle
import os
import zipfile

def download_data_from_kaggle():
    try:
        from google.colab import files
        uploaded = files.upload()
    except ImportError:
        print("Running outside of Colab. Please ensure your kaggle.json is in ~/.kaggle/")

    if 'kaggle.json' in os.listdir('.'):
        !mkdir -p ~/.kaggle
        !mv kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
    else:
        print("kaggle.json not found. Please upload it or place it in the correct directory.")

    if not os.path.exists('cpe342-karena.zip'):
        print("Downloading data from Kaggle competition 'cpe342-karena'...")
        !kaggle competitions download -c cpe342-karena
    else:
        print("Data already downloaded.")

    if os.path.exists('cpe342-karena.zip'):
        print("Unzipping data...")
        try:
            with zipfile.ZipFile('cpe342-karena.zip', 'r') as zip_ref:
                zip_ref.extractall('.')
            print("Data unzipped.")
        except zipfile.BadZipFile:
            print("Error: Downloaded file is not a valid zip file.")
        except Exception as e:
            print(f"An error occurred during unzipping: {e}")
    else:
        print("Zip file not found, cannot unzip.")

In [2]:
download_data_from_kaggle()

Saving kaggle.json to kaggle.json
 97% 898M/922M [00:08<00:00, 227MB/s]
100% 922M/922M [00:08<00:00, 118MB/s]
Unzipping data...
Data unzipped.


#Task5: Account Security Monitoring

In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# 1. Load Data
try:
    df = pd.read_csv('public_dataset/task5/test.csv')
    print("Data loaded.")
except FileNotFoundError:
    print("Error: test.csv not found.")
    exit()

# 2. Advanced Feature Engineering
# สร้าง Feature ใหม่ที่สรุปพฤติกรรมจาก 4 ช่วงเวลา (_1 ถึง _4)
# ดึงชื่อกลุ่ม Feature หลักๆ (ตัด suffix ออก)
base_features = set()
for col in df.columns:
    if col.endswith('_1'):
        base_features.add(col[:-2])

X_eng = df.copy()

# วนลูปสร้างค่า Mean และ Std สำหรับแต่ละกลุ่ม Feature
for base in base_features:
    cols = [f"{base}_{i}" for i in range(1, 5)]
    # เช็คว่ามีคอลัมน์ครบไหม
    if all(c in df.columns for c in cols):
        # ค่าเฉลี่ย (บอกระดับพฤติกรรมทั่วไป)
        X_eng[f'{base}_mean_agg'] = df[cols].mean(axis=1)
        # ค่าเบี่ยงเบนมาตรฐาน (บอกความนิ่ง/ความผันผวน) - สำคัญมากสำหรับจับบอทหรือการแฮก
        X_eng[f'{base}_std_agg'] = df[cols].std(axis=1)
        # ค่าผลต่างสูงสุด (Max Drop/Jump)
        X_eng[f'{base}_range_agg'] = df[cols].max(axis=1) - df[cols].min(axis=1)

# เลือกเฉพาะคอลัมน์ตัวเลขสำหรับเข้าโมเดล
X = X_eng.select_dtypes(include=[np.number])

# Clean & Scale
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# 3. Model Ensemble Strategy
# เพื่อเพิ่ม F3 (Recall) เราจะใช้ 2 โมเดลช่วยกันโหวต

# Model A: Isolation Forest
# contamination=0.15 หมายถึงเราคาดการณ์ว่ามีคนร้ายประมาณ 15% (ตั้งเผื่อไว้เพื่อเพิ่ม Recall)
iso = IsolationForest(n_estimators=300, contamination=0.15, random_state=42, n_jobs=-1)
iso.fit(X_scaled)
# ได้คะแนนความผิดปกติ (ยิ่งต่ำยิ่งผิดปกติ)
iso_scores = iso.decision_function(X_scaled)

# Model B: PCA Reconstruction Error (คล้าย Autoencoder)
# บีบอัดข้อมูลเหลือครึ่งหนึ่ง แล้วขยายกลับ
n_components = int(X_scaled.shape[1] * 0.8)
pca = PCA(n_components=n_components, random_state=42)
X_pca = pca.fit_transform(X_scaled)
X_reconstructed = pca.inverse_transform(X_pca)
# คำนวณ Error: ถ้ากู้คืนไม่ได้ แสดงว่าเป็น Outlier
reconstruction_error = np.mean(np.square(X_scaled - X_reconstructed), axis=1)

# 4. Combine Scores (Rank Averaging)
# แปลงคะแนนให้อยู่ในสเกลเดียวกันด้วยการจัดอันดับ (Ranking)
# IsoScore: ค่าต่ำ = Anomaly -> Rank น้อย = Anomaly มาก
iso_rank = np.argsort(iso_scores) # index ของคนที่ปกติ -> ผิดปกติมาก
# สร้าง rank array
iso_rank_val = np.zeros(len(iso_scores))
iso_rank_val[iso_rank] = np.arange(len(iso_scores))

# RecError: ค่าสูง = Anomaly -> Rank มาก = Anomaly มาก
pca_rank = np.argsort(reconstruction_error)
pca_rank_val = np.zeros(len(reconstruction_error))
pca_rank_val[pca_rank] = np.arange(len(reconstruction_error))

# รวมพลัง: เราจะมองหาคนที่ "ผิดปกติ" ในทั้งสองโมเดล
# เนื่องจาก IsoForest (ค่าต่ำ=แย่) และ PCA (ค่าสูง=แย่) เราต้องกลับด้าน Iso ให้ทิศทางเดียวกัน
# หรือใช้ง่ายกว่านั้น: Normalize Scores
from sklearn.preprocessing import MinMaxScaler
scaler_mm = MinMaxScaler()
s1 = scaler_mm.fit_transform(iso_scores.reshape(-1, 1)).flatten() # 0=Bad, 1=Good (โดยประมาณของ Iso)
s1 = 1 - s1 # กลับด้านเป็น 1=Bad (Anomaly), 0=Good

s2 = scaler_mm.fit_transform(reconstruction_error.reshape(-1, 1)).flatten() # 1=Bad, 0=Good

# Average Score
final_score = (s1 + s2) / 2

# 5. Thresholding for High Recall (F3)
# เราจะตัดที่ Top 15-20% ของคะแนนที่สูงที่สุดว่าเป็น Anomaly
threshold_percentile = 85 # ตัดที่ Percentile 85 (คือเอา Top 15% เป็นคนร้าย)
threshold_val = np.percentile(final_score, threshold_percentile)

final_predictions = [1 if x >= threshold_val else 0 for x in final_score]

# Create Submission
submission = df.copy()
submission['is_anomaly'] = final_predictions

# บันทึกไฟล์
submission.to_csv('submission.csv', index=False)

print(f"Generated submission.csv with Ensemble approach.")
print(f"Total Anomalies Flagged: {sum(final_predictions)} ({sum(final_predictions)/len(final_predictions)*100:.2f}%)")

Data loaded.
Generated submission.csv with Ensemble approach.
Total Anomalies Flagged: 4401 (17.00%)
